In [8]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import os
import warnings
from sklearn.model_selection import cross_validate

# 1. Importar Normalización Ganadora
from normalization_functions import pipeline_d_normalize
# 2. Importar Características Aumentadas 
from lexicon_functions import get_augmented_features

In [9]:

# --- 1. Configuración de Rutas ---
TRAIN_PATH = os.path.join("..", "data", "processed", "train.csv")
RESULTS_PATH = os.path.join("..", "results", "fase_3_mlp_results.csv")

os.makedirs(os.path.dirname(RESULTS_PATH), exist_ok=True)
warnings.filterwarnings("ignore", category=UserWarning)

# --- 2. Cargar y Preparar Datos ---
print(f"Iniciando Fase 3: MLP + GridSearchCV")
try:
    df_train = pd.read_csv(TRAIN_PATH)
    print(f"Datos de entrenamiento cargados desde '{TRAIN_PATH}'.")
except FileNotFoundError:
    print(f"\n¡Error de Archivo! No se encontró '{TRAIN_PATH}'.")
    exit()

print("Aplicando 'pipeline_d_normalize' (Combinación Ganadora) a los datos...")
# Aplicar la normalización GANADORA
X_train = df_train['text'].apply(pipeline_d_normalize)
y_train = df_train['Polarity']
print(f"Datos normalizados listos. {len(X_train)} documentos.")


# Definir la rejilla de hiperparámetros para MLP
# 'classifier__' es el prefijo para aplicar los parámetros al paso 'classifier' del pipeline
FIXED_PARAMS = {
    'hidden_layer_sizes': (50,),
    'alpha': 0.01,
    'activation': 'relu',
    'solver': 'adam',
    'max_iter': 500,
    'random_state': 0,
    'early_stopping': True
}

print(f"Entrenando MLP con parámetros fijos: {FIXED_PARAMS}")

# --- 3. Definir Pipeline y Rejilla de Búsqueda ---

# Crear el Pipeline con la combinación GANADORA
text_pipeline = Pipeline([
    ('features', get_augmented_features()), 
    ('classifier', MLPClassifier(**FIXED_PARAMS))
])


Iniciando Fase 3: MLP + GridSearchCV
Datos de entrenamiento cargados desde '..\data\processed\train.csv'.
Aplicando 'pipeline_d_normalize' (Combinación Ganadora) a los datos...
Datos normalizados listos. 24169 documentos.
Entrenando MLP con parámetros fijos: {'hidden_layer_sizes': (50,), 'alpha': 0.01, 'activation': 'relu', 'solver': 'adam', 'max_iter': 500, 'random_state': 0, 'early_stopping': True}


In [10]:

# --- 4. Configurar y Ejecutar Validación Cruzada ---

print("\n--- INICIANDO VALIDACIÓN CRUZADA (5 folds) ---")

cv_results = cross_validate(
    text_pipeline, 
    X_train, 
    y_train, 
    cv=5, 
    scoring='f1_macro',
    n_jobs=-1,
    verbose=10
)

mean_score = cv_results['test_score'].mean()
std_score = cv_results['test_score'].std()

print(f"\n>>> Resultado Final (Fast Track): {mean_score:.4f} (+/- {std_score:.4f}) <<<")
# Opción A: Guardar el detalle de los 5 folds (útil para análisis profundo)
# Convertimos el diccionario a DataFrame primero
df_folds = pd.DataFrame(cv_results)
# df_folds.to_csv("results_folds_detail.csv", index=False) # Opcional

# Opción B: Guardar el RESUMEN (Lo que necesitas para comparar con tu tabla final)
# Creamos un DataFrame manual con la estructura de tus reportes anteriores
summary_df = pd.DataFrame({
    "model": ["MLPClassifier (Fast Track)"],
    "features": ["Pipeline D + Binary + Lexicons"],
    "avg_f1_macro": [mean_score],
    "std_f1_macro": [std_score],
    "params": [str(FIXED_PARAMS)] 
})

summary_df.to_csv(RESULTS_PATH, index=False)

print(f"Resumen de resultados guardado exitosamente en '{RESULTS_PATH}'")


--- INICIANDO VALIDACIÓN CRUZADA (5 folds) ---


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 